In [2]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from os import listdir
from os.path import isfile, join
from keras.callbacks import CSVLogger

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [3]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)
    data.append(dataset[indices])

    if single_step:
      labels.append(target[i+target_size])
    else:
      labels.append(target[i:i+target_size])

  return np.array(data), np.array(labels)

In [6]:
mypath = "D:/HomeTask/Dataset/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [ ]:
for file in onlyfiles[1:]:
    print(file)
    df = pd.read_csv(mypath+file, sep = '\t')
    df['Date Time'] = df['<DATE>'] + df['<TIME>']
    df = df.drop(labels = 0)
    features_considered = ['<OPEN>','<HIGH>','<LOW>','<CLOSE>']
    features = df[features_considered]
    features.index = df['Date Time']
    dataset = features.values
    TRAIN_SPLIT = round(0.7*df.shape[0])
    data_mean = dataset[:TRAIN_SPLIT].mean(axis=0)
    data_std = dataset[:TRAIN_SPLIT].std(axis=0)
    dataset = (dataset-data_mean)/data_std
    
    past_history = 250
    future_target = 0
    STEP = 1
    x_train_single, y_train_single = multivariate_data(dataset, dataset[:, 1], 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=True)
    x_val_single, y_val_single = multivariate_data(dataset, dataset[:, 1],
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=True)
    
    BATCH_SIZE = 1000
    BUFFER_SIZE = 10000
    train_data_single = tf.data.Dataset.from_tensor_slices((x_train_single, y_train_single))
    train_data_single = train_data_single.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
    val_data_single = tf.data.Dataset.from_tensor_slices((x_val_single, y_val_single))
    val_data_single = val_data_single.batch(BATCH_SIZE).repeat()
    
    single_step_model = tf.keras.models.Sequential()
    single_step_model.add(tf.keras.layers.LSTM(250,
                                           input_shape=x_train_single.shape[-2:], 
                                           kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    single_step_model.add(tf.keras.layers.Dense(1))
    single_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='mean_squared_error')
    
    os.makedirs(mypath+file[:-4], mode=0o777, exist_ok=True)
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=mypath+file[:-4]+'/cp-{epoch:04d}.ckpt',
                                                 save_weights_only=True,
                                                 verbose=1)
    
    csv_logger = CSVLogger(mypath+file[:-4]+'/log.csv', append=True, separator=';')
    EVALUATION_INTERVAL = 250
    EPOCHS = 100
    single_step_history = single_step_model.fit(train_data_single, epochs=EPOCHS,
                                            steps_per_epoch=EVALUATION_INTERVAL,
                                            validation_data=val_data_single,
                                            validation_steps=50,
                                            callbacks=[cp_callback, csv_logger])
    print('On ', file, 'data sucsessfuly learned')
    x = single_step_model.predict(x_val_single)
    counter = 0
    for i in range(1, x.shape[0]):
        if (y_val_single[i] - y_val_single[i-1]) * (x[i] - x[i-1]) > 0:
            counter = counter +1
    print('On ', file, 'data accuracy = ', counter/x.shape[0])
    with open(mypath+file[:-4]+'/log.csv','a') as fd:
        fd.write(str(counter/x.shape[0]))

ADAUSDT_M15.csv
Epoch 1/100
250/250 [==============================] - ETA: 0s - loss: 0.0244
Epoch 1: saving model to D:/HomeTask/Dataset/ADAUSDT_M15\cp-0001.ckpt
250/250 [==============================] - 16s 60ms/step - loss: 0.0244 - val_loss: 0.0063
Epoch 2/100
250/250 [==============================] - ETA: 0s - loss: 0.0064
Epoch 2: saving model to D:/HomeTask/Dataset/ADAUSDT_M15\cp-0002.ckpt
250/250 [==============================] - 15s 59ms/step - loss: 0.0064 - val_loss: 0.0089
Epoch 3/100
250/250 [==============================] - ETA: 0s - loss: 0.0038
Epoch 3: saving model to D:/HomeTask/Dataset/ADAUSDT_M15\cp-0003.ckpt
250/250 [==============================] - 15s 59ms/step - loss: 0.0038 - val_loss: 0.0032
Epoch 4/100
250/250 [==============================] - ETA: 0s - loss: 0.0029
Epoch 4: saving model to D:/HomeTask/Dataset/ADAUSDT_M15\cp-0004.ckpt
250/250 [==============================] - 15s 59ms/step - loss: 0.0029 - val_loss: 5.8504e-04
Epoch 5/100
250/250 [===

In [7]:
onlyfiles

['ADAUSDT_H1.csv',
 'ADAUSDT_M15.csv',
 'ANKRUSDT_H1.csv',
 'ANKRUSDT_M15.csv',
 'ATOMUSDT_H1.csv',
 'ATOMUSDT_M15.csv',
 'AVAXUSDT_H1.csv',
 'AVAXUSDT_M15.csv',
 'AXSUSDT_H1.csv',
 'AXSUSDT_M15.csv',
 'BNBUSDT_H1.csv',
 'BNBUSDT_M15.csv',
 'BTCUSDT_H1.csv',
 'BTCUSDT_M15.csv',
 'CHZUSDT_H1.csv',
 'CHZUSDT_M15.csv',
 'DOGEUSDT_H1.csv',
 'DOGEUSDT_M15.csv',
 'DOTUSDT_H1.csv',
 'DOTUSDT_M15.csv',
 'EGLDUSDT_H1.csv',
 'EGLDUSDT_M15.csv',
 'ENJUSDT_H1.csv',
 'ENJUSDT_M15.csv',
 'ETHUSDT_H1.csv',
 'ETHUSDT_M15.csv',
 'FTMUSDT_H1.csv',
 'FTMUSDT_M15.csv',
 'GALAUSDT_H1.csv',
 'GALAUSDT_M15.csv',
 'HBARUSDT_H1.csv',
 'HBARUSDT_M15.csv',
 'HNTUSDT_H1.csv',
 'HNTUSDT_M15.csv',
 'ICPUSDT_H1.csv',
 'ICPUSDT_M15.csv',
 'IOTXUSDT_H1.csv',
 'IOTXUSDT_M15.csv',
 'LINKUSDT_H1.csv',
 'LINKUSDT_M15.csv',
 'LRCUSDT_H1.csv',
 'LRCUSDT_M15.csv',
 'LTCUSDT_H1.csv',
 'LTCUSDT_M15.csv',
 'MANAUSDT_H1.csv',
 'MANAUSDT_M15.csv',
 'MATICUSDT_H1.csv',
 'MATICUSDT_M15.csv',
 'NEARUSDT_H1.csv',
 'NEARUSDT_M15.csv',

###### 